In [33]:
%pip install faker



   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   -------------------------------------- - 1.8/1.9 MB 14.3 MB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 13.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\reaga\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


# Data Preprocess


## Import Libraries


In [45]:

# # Data Preprocess for Category Classificatin
import pandas as pd
from sklearn.model_selection import train_test_split
import string
from collections import Counter
import re
import random
from faker import Faker



## Initialize Dataset


In [46]:
file_path = '../dataset/dataset.csv'


In [47]:

# Load dataset from CSV file
df = pd.read_csv(file_path)

df = df.drop(columns=['flags'])

# Display the first few rows of the dataframe
df.head()


,instruction,category,intent,response
0,question about cancelling order {{Order Number}},ORDER,cancel_order,I've understood you have a question regarding ...
1,i have a question about cancelling oorder {{Or...,ORDER,cancel_order,I've been informed that you have a question ab...
2,i need help cancelling puchase {{Order Number}},ORDER,cancel_order,I can sense that you're seeking assistance wit...
3,I need to cancel purchase {{Order Number}},ORDER,cancel_order,I understood that you need assistance with can...
4,"I cannot afford this order, cancel purchase {{...",ORDER,cancel_order,I'm sensitive to the fact that you're facing f...


In [48]:
# Check the total length of the dataframe
total_length = len(df)
print(f"Total length of the dataframe: {total_length}")

# Check for any missing or empty values in each column
missing_values = df.isnull().sum()
print("Missing values in each column:")
print(missing_values)

# Check for any missing or empty values in each row
missing_values_rows = df.isnull().sum(axis=1)
print("Missing values in each row:")
print(missing_values_rows)


Total length of the dataframe: 26872
Missing values in each column:
instruction    0
category       0
intent         0
response       0
dtype: int64
Missing values in each row:
0        0
1        0
2        0
3        0
4        0
        ..
26867    0
26868    0
26869    0
26870    0
26871    0
Length: 26872, dtype: int64


## Checking of Dataset


In [49]:
unique_intents = df['intent'].unique()
unique_categories = df['category'].unique()

print("Unique intents:")
print(unique_intents)
print("Total unique intents:", len(unique_intents))

print("\nUnique categories:")
print(unique_categories)
print("Total unique categories:", len(unique_categories))


Unique intents:
['cancel_order' 'change_order' 'change_shipping_address'
 'check_cancellation_fee' 'check_invoice' 'check_payment_methods'
 'check_refund_policy' 'complaint' 'contact_customer_service'
 'contact_human_agent' 'create_account' 'delete_account'
 'delivery_options' 'delivery_period' 'edit_account' 'get_invoice'
 'get_refund' 'newsletter_subscription' 'payment_issue' 'place_order'
 'recover_password' 'registration_problems' 'review'
 'set_up_shipping_address' 'switch_account' 'track_order' 'track_refund']
Total unique intents: 27

Unique categories:
['ORDER' 'SHIPPING' 'CANCEL' 'INVOICE' 'PAYMENT' 'REFUND' 'FEEDBACK'
 'CONTACT' 'ACCOUNT' 'DELIVERY' 'SUBSCRIPTION']
Total unique categories: 11


## Category and Intent


Intent


In [50]:
# Get the unique intents
unique_intents_list = df['intent'].unique()

# Print the unique intents
print("Unique intents:")
print(unique_intents_list)

# Get the distribution of intents
intent_distribution = df['intent'].value_counts()

# Print the distribution of intents
print("\nIntent distribution:")
print(intent_distribution)

print("Total unique intents:", len(unique_intents_list))


Unique intents:
['cancel_order' 'change_order' 'change_shipping_address'
 'check_cancellation_fee' 'check_invoice' 'check_payment_methods'
 'check_refund_policy' 'complaint' 'contact_customer_service'
 'contact_human_agent' 'create_account' 'delete_account'
 'delivery_options' 'delivery_period' 'edit_account' 'get_invoice'
 'get_refund' 'newsletter_subscription' 'payment_issue' 'place_order'
 'recover_password' 'registration_problems' 'review'
 'set_up_shipping_address' 'switch_account' 'track_order' 'track_refund']

Intent distribution:
intent
edit_account                1000
switch_account              1000
check_invoice               1000
complaint                   1000
contact_customer_service    1000
delivery_period              999
registration_problems        999
check_payment_methods        999
contact_human_agent          999
payment_issue                999
newsletter_subscription      999
get_invoice                  999
place_order                  998
cancel_order        

Category


In [51]:
# Get the unique categories
unique_categories_list = df['category'].unique()

# Print the unique categories
print("Unique categories:")
print(unique_categories_list)

# Get the distribution of categories
category_distribution = df['category'].value_counts()

# Print the distribution of categories
print("\nCategory distribution:")
print(category_distribution)

print("Total unique categories:", len(unique_categories_list))


Unique categories:
['ORDER' 'SHIPPING' 'CANCEL' 'INVOICE' 'PAYMENT' 'REFUND' 'FEEDBACK'
 'CONTACT' 'ACCOUNT' 'DELIVERY' 'SUBSCRIPTION']

Category distribution:
category
ACCOUNT         5986
ORDER           3988
REFUND          2992
INVOICE         1999
CONTACT         1999
PAYMENT         1998
FEEDBACK        1997
DELIVERY        1994
SHIPPING        1970
SUBSCRIPTION     999
CANCEL           950
Name: count, dtype: int64
Total unique categories: 11


In [52]:
# Convert the text in the 'instruction' and 'intent' columns to lowercase
df['instruction'] = df['instruction'].str.lower()
df['intent'] = df['intent'].str.lower()

# Display the resulting DataFrame
print(df)



                                             instruction category  \
0       question about cancelling order {{order number}}    ORDER   
1      i have a question about cancelling oorder {{or...    ORDER   
2        i need help cancelling puchase {{order number}}    ORDER   
3             i need to cancel purchase {{order number}}    ORDER   
4      i cannot afford this order, cancel purchase {{...    ORDER   
...                                                  ...      ...   
26867  i am waiting for a rebate of {{refund amount}}...   REFUND   
26868  how to see if there is anything wrong with my ...   REFUND   
26869  i'm waiting for a reimbjrsement of {{currency ...   REFUND   
26870  i don't know what to do to see my reimbursemen...   REFUND   
26871  i need to know if there is anything new on the...   REFUND   

             intent                                           response  
0      cancel_order  I've understood you have a question regarding ...  
1      cancel_order  I've

## Entity


### Placeholder


Checking placeholder


In [55]:
import pandas as pd
import re
from collections import Counter

# Detect rows with placeholders
df['has_placeholder'] = df['instruction'].str.contains(r'\{\{.*?\}\}')

# Count rows with placeholders
rows_with_placeholders = df['has_placeholder'].sum()

# Extract placeholders function
def extract_placeholders(text):
    return re.findall(r'\{\{(.*?)\}\}', text)

# Apply function and get all placeholders
placeholders = df[df['has_placeholder']]['instruction'].apply(extract_placeholders).explode()

# Count occurrences of each placeholder
placeholder_counts = Counter(placeholders)

# Convert to DataFrame
placeholder_df = pd.DataFrame.from_dict(placeholder_counts, orient='index', columns=['count'])

# Display initial results
print(f"Rows with placeholders: {rows_with_placeholders}")
print("\nInitial Distribution of Placeholders:")
print(placeholder_df.sort_values(by='count', ascending=False))

# Print total rows and missing values
print("\nTotal rows in the dataset:", len(df))
print("\nMissing values per column:")
print(df.isnull().sum())

# Remove placeholders with the lowest occurrence
if not placeholder_df.empty:
    min_count = placeholder_df['count'].min()  # Find the lowest count
    placeholder_df = placeholder_df[placeholder_df['count'] > min_count]  # Keep only placeholders with higher counts

# Display updated results
print("\nUpdated Distribution of Placeholders (After Removing Lowest Occurrences):")
print(placeholder_df.sort_values(by='count', ascending=False))


Rows with placeholders: 0

Initial Distribution of Placeholders:
Empty DataFrame
Columns: [count]
Index: []

Total rows in the dataset: 26872

Missing values per column:
instruction        0
category           0
intent             0
response           0
has_placeholder    0
ner_labels         0
dtype: int64

Updated Distribution of Placeholders (After Removing Lowest Occurrences):
Empty DataFrame
Columns: [count]
Index: []


In [54]:
import re
import pandas as pd
from faker import Faker

# Initialize Faker
fake = Faker()

# Define synthetic data generators
data_generators = {
    'order number': lambda: fake.unique.bothify(text='ORD-#####'),
    'account type': lambda: fake.random_element(elements=('Business', 'Personal')),
    'person name': lambda: fake.name(),
    'account category': lambda: fake.random_element(elements=('Category A', 'Category B', 'Category C')),

    'refund amount': lambda: f"${fake.random_number(digits=3)}.00",
    'currency symbol': lambda: fake.currency_symbol(),
    'delivery city': lambda: fake.city(),
    'delivery country': lambda: fake.country(),
    'invoice number': lambda: fake.unique.bothify(text='INV###')
}

# Function to replace placeholders and track NER labels
def replace_placeholders(text, data_generators):
    if pd.isnull(text):
        return text, []  # Return empty list for NER labels

    placeholder_pattern = re.compile(r'\{{2}(.+?)\}{2}')  # Match placeholders {{...}}
    placeholders = placeholder_pattern.findall(text)

    ner_labels = []  # Store entity labels

    for placeholder in placeholders:
        placeholder_cleaned = placeholder.strip().lower()  # Normalize case
        if placeholder_cleaned in data_generators:
            replacement_value = data_generators[placeholder_cleaned]()  # Generate synthetic data
            text = text.replace(f'{{{{{placeholder}}}}}', replacement_value, 1)
            
            # Append entity info in required format
            ner_labels.append({'text': replacement_value, 'label': placeholder_cleaned.replace(" ", "_")})

    return text, ner_labels

# Apply function
df[['instruction', 'ner_labels']] = df['instruction'].apply(lambda x: pd.Series(replace_placeholders(x, data_generators)))

# Save processed dataset
output_csv_path = '../dataset/processed_dataset.csv'
df.to_csv(output_csv_path, index=False)

print(f"✅ Placeholders replaced and dataset saved to {output_csv_path}.")
print(df.head())


✅ Placeholders replaced and dataset saved to ../dataset/processed_dataset.csv.
                                         instruction category        intent  \
0          question about cancelling order ORD-71383    ORDER  cancel_order   
1  i have a question about cancelling oorder ORD-...    ORDER  cancel_order   
2           i need help cancelling puchase ORD-48300    ORDER  cancel_order   
3                i need to cancel purchase ORD-65724    ORDER  cancel_order   
4  i cannot afford this order, cancel purchase OR...    ORDER  cancel_order   

                                            response  has_placeholder  \
0  I've understood you have a question regarding ...             True   
1  I've been informed that you have a question ab...             True   
2  I can sense that you're seeking assistance wit...             True   
3  I understood that you need assistance with can...             True   
4  I'm sensitive to the fact that you're facing f...             True   

       